In [ ]:
#注意事項:
# 現在可以在Cell裡，按下 按下 <Shift> + <Enter> 執行Cell裡的程式碼, <Enter> 是換到Cell裡的下一行
# 如果要關閉Notebook，請選擇選單: File > Close and Halt    才能確實停止目前正在執行的程式，並且釋放資源

# Basic RDD[T] Operation

## create basic RDD

In [ ]:
intRDD = sc.parallelize([3,1, 2, 5, 5])
intRDD.collect()

In [ ]:
stringRDD = sc.parallelize(["Apple", "Orange", "Banana","Grape","Apple"])
stringRDD.collect()

In [ ]:
fileRDD = sc.textFile("/opt/spark/README.md")
fileRDD.collect()

## Basic RDD Transformation 

### map()

In [ ]:
def addOne(x):
    return (x+1)

intRDD.map(addOne).collect()

In [ ]:
intRDD.map(lambda  x : x + 1).collect()

In [ ]:
intRDD.map(lambda x: range(1,x+1)).collect()

In [ ]:
stringRDD.map(lambda x: "fruit:"+x).collect()

### filter()

In [ ]:
intRDD.filter(lambda x : x < 3).collect()

In [ ]:
intRDD.filter(lambda x : x == 3).collect()

In [ ]:
intRDD.filter(lambda x : 1< x and x <5  ).collect()

In [ ]:
intRDD.filter(lambda x : x >=5 or x<3 ).collect()

In [ ]:
stringRDD.filter(lambda x : "ra" in x ).collect()

### flatmap()

In [ ]:
intRDD.flatMap(lambda x: range(1,x+1)).collect()

## Basic RDD Action

### 讀取元素

In [ ]:
intRDD.first()

In [ ]:
intRDD.take(2)

### 常用統計量

In [ ]:
intRDD.stats()

In [ ]:
intRDD.min()

In [ ]:
intRDD.max()

In [ ]:
intRDD.stdev()

In [ ]:
intRDD.count()

In [ ]:
intRDD.sum()

In [ ]:
intRDD.mean()

### reduce()

In [ ]:
intRDD.reduce(lambda a, b: a + b)

### collect()

In [ ]:
intRDD.collect()

### saveAsTextFile()

In [ ]:
% rm -R output

In [ ]:
intRDD.saveAsTextFile("output")

# PairRDD[K, V] Operation

## create PairRDD

In [ ]:
kvRDD1 = sc.parallelize([(3, 4), (3, 6), (5, 6), (1, 2)])
kvRDD1.collect()

In [ ]:
kvRDD1.keys().collect()

In [ ]:
kvRDD1.values().collect()

## PairRDD use RDD operation

### filter() / first() / take()

In [ ]:
kvRDD1.filter(lambda keyValue: keyValue[0] < 5).collect()

In [ ]:
kvRDD1.filter(lambda keyValue: keyValue[1] < 5).collect()

In [ ]:
kvRDD1.first()

In [ ]:
kvRDD1.take(2)

## PairRDD Transformation

In [ ]:
kvRDD1.mapValues(lambda x : x * x).collect()

In [ ]:
kvRDD1.reduceByKey(lambda x ,y: x+y).collect()

## PairRDD Action

In [ ]:
kvRDD1.countByKey()

In [ ]:
KV=kvRDD1.collectAsMap()
KV

In [ ]:
type(KV)

In [ ]:
KV[3]

In [ ]:
KV[1]

In [ ]:
kvRDD1.lookup(3)

In [ ]:
kvRDD1.lookup(5)

# Lab2-1: 使用Spark 建立WordCount 

In [ ]:
%rm -R output

In [ ]:
%pwd

In [ ]:
textFile = sc.textFile("/opt/spark/README.md")
stringRDD=textFile.flatMap(lambda line : line.split(" "))
countsRDD = stringRDD.map(lambda word : (word, 1)).reduceByKey(lambda x,y : x+y)
countsRDD.saveAsTextFile("output")

In [ ]:
%ll output

In [ ]:
%cat output/part-00000

In [ ]:
%cat output/part-00001

# Lab2-2: 使用Spark 建立k-means

In [ ]:
import numpy as np

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
import random

In [ ]:
#define utility functions

def parseVector2(tuple):
    return np.array([tuple[0], tuple[1]])

def closestPoint(p, centers):
    bestIndex = 0
    closest = float("+inf")
    for i in range(len(centers)):
        tempDist = np.sum((p - centers[i]) ** 2)
        if tempDist < closest:
            closest = tempDist
            bestIndex = i
    return bestIndex

In [ ]:
# create an RDD of 90 random vectors
x1 = [random.uniform(-3,-1) for i in range(30)]
y1 = [random.uniform(-0.5,0.5) for i in range(30)]

x2 = [random.uniform(-1,1) for i in range(30)]
y2 = [random.uniform(0.5,1.5) for i in range(30)]

x3 = [random.uniform(1,3) for i in range(30)]
y3 = [random.uniform(-0.5,0.5) for i in range(30)]

plt.axis([-5, 5, -1, 2])
plt.plot(x1,y1,'r*')
plt.plot(x2,y2,'bs')
plt.plot(x3,y3,'gx')

In [ ]:
# create data RDD
p1x = sc.parallelize(x1)
p1y = sc.parallelize(y1)
p1 = p1x.zip(p1y)

p2x = sc.parallelize(x2)
p2y = sc.parallelize(y2)
p2 = p2x.zip(p2y)

p3x = sc.parallelize(x3)
p3y = sc.parallelize(y3)
p3 = p3x.zip(p3y)

data = p1.union(p2).union(p3).map(parseVector2).cache()

In [ ]:
K = 3
convergeDist = 0.01

kPoints = data.takeSample(False, K, 2)
tempDist = 1.0

kPoints

In [ ]:
#Main loop
while tempDist > convergeDist:
    closest = data.map(lambda p: (closestPoint(p, kPoints), (p, 1)))
    pointStats = closest.reduceByKey(lambda p1_c1, p2_c2: (p1_c1[0] + p2_c2[0], p1_c1[1] + p2_c2[1]))
    newPoints = pointStats.map(lambda st: (st[0], st[1][0] / st[1][1])).collect()

    tempDist = sum(np.sum((kPoints[iK] - p) ** 2) for (iK, p) in newPoints)

    for (iK, p) in newPoints:
        kPoints[iK] = p


print("Final centers: " + str(kPoints))

In [ ]:
#plot result
plt.axis([-5, 5, -1, 2])
plt.plot(x1,y1,'r*')
plt.plot(x2,y2,'bs')
plt.plot(x3,y3,'gx')

plt.plot(kPoints[0][0],kPoints[0][1],'yo')
plt.plot(kPoints[1][0],kPoints[1][1],'yo')
plt.plot(kPoints[2][0],kPoints[2][1],'yo')